In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def make_coordinates(image,line_parameters):
    slope,intercept=line_parameters
    y1=int(image.shape[0])
    y2=int(y1*(3/5))
    x1=int((y1-intercept)/slope)
    x2=int((y2-intercept)/slope)
    return np.array([x1,y1,x2,y2])


def average_slope_intercept(image,lines):
    #Left_fit list for storing slope and intercept of left side multiple lines.
    #Right_fit list for storing slope and intercept of Right side multiple lines.
    left_fit=[]
    right_fit=[]
    for line in lines:
        x1,y1,x2,y2=line.reshape(4)
        #polyfit will take x1,x2,y1,y2 as an input and fit a polynomial of degree 1
        #It will give an output as slope and intercept
        parameters=np.polyfit((x1,x2),(y1,y2),1)
        slope=parameters[0]
        intercept=parameters[1]
        #Negative slope corresponds to left side of line since its y value is decreasing
        #so all negative slope value will append in left side list and same for right side
        if slope<0:
            left_fit.append((slope,intercept))
        else:
            right_fit.append((slope,intercept))
    #We received many slope and intercept value since we have multiple lines
    #so we average out the slope and intercept value on axis 0
    left_fit_average=np.average(left_fit,axis=0)
    right_fit_average=np.average(right_fit,axis=0)
    #Use this averaged slope and intercept value to make a line equation for right and left side 
    left_line=make_coordinates(image,left_fit_average)
    right_line=make_coordinates(image,right_fit_average)
    averaged_lines = [left_line, right_line]
    return averaged_lines

def canny(image):
    lane_image=np.copy(image)
    gray=cv2.cvtColor(lane_image,cv2.COLOR_RGB2GRAY)
    blur=cv2.GaussianBlur(gray,(5,5),0)
    canny=cv2.Canny(blur,50,150)
    return canny

def region_of_interest(image):
    height=image.shape[0]
    triangle=np.array([
        [(200,height),(1100,height),(550,250)]
        ])
    mask=np.zeros_like(image)
    cv2.fillPoly(mask,triangle,255)
    masked_image=cv2.bitwise_and(image,mask)
    return masked_image

def display_lines(image,lines):
    line_image=np.zeros_like(image)
    if lines is not None:
        for line in lines:
            x1,y1,x2,y2=line.reshape(4)
            cv2.line(line_image,(x1,y1),(x2,y2),(255,0,0),10)
    return line_image

# image=cv2.imread("test_image.jpg")
# lane_image=np.copy(image)
# canny_image=canny(lane_image)
# interest_image=region_of_interest(canny_image)
# cropped_image=region_of_interest(canny_image)
# lines=cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
# averaged_lines=average_slope_intercept(lane_image,lines)
# line_image=display_lines(lane_image,averaged_lines)
# combo_image=cv2.addWeighted(lane_image,0.8,line_image,1,1)

#plt.imshow(canny_image)
#plt.show()

# cv2.imshow("result",line_image)
# cv2.waitKey(0)

cap=cv2.VideoCapture("test2.mp4")
while(cap.isOpened()):
    _, frame=cap.read()
    canny_image=canny(frame)
    interest_image=region_of_interest(canny_image)
    cropped_image=region_of_interest(canny_image)
    lines=cv2.HoughLinesP(cropped_image,2,np.pi/180,100,np.array([]),minLineLength=40,maxLineGap=5)
    averaged_lines=average_slope_intercept(frame,lines)
    line_image=display_lines(frame,averaged_lines)
    combo_image=cv2.addWeighted(frame,0.8,line_image,1,1)
    cv2.imshow("result",combo_image)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()